In [ ]:
# Define the output file name
output_file = "consolidated_data.csv"

# Initialize a list to store merged DataFrames from all subfolders
all_subfolder_data = []

# Traverse through all subfolders in the current directory
for root, dirs, files in os.walk(os.getcwd()):
    infotable_df = None
    coverpage_df = None
    print(root)

    for file in files:
        file_path = os.path.join(root, file)

        # Process INFOTABLE.tsv
        if file == "INFOTABLE.tsv":
            try:
                print(f"Processing: {file_path}")
                infotable_df = pd.read_csv(file_path, sep="\t", low_memory=False)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

        # Process COVERPAGE.tsv
        if file == "COVERPAGE.tsv":
            try:
                print(f"Processing: {file_path}")
                coverpage_df = pd.read_csv(file_path, sep="\t", low_memory=False)
                # Keep only necessary columns
                coverpage_df = coverpage_df[["ACCESSION_NUMBER", "FILINGMANAGER_NAME", "DATEREPORTED"]]
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    # Merge INFOTABLE and COVERPAGE on ACCESSION_NUMBER if both are present
    if infotable_df is not None and coverpage_df is not None:
        try:
            merged_subfolder_data = pd.merge(infotable_df, coverpage_df, on="ACCESSION_NUMBER", how="inner")
            all_subfolder_data.append(merged_subfolder_data)
        except Exception as e:
            print(f"Error merging files in {root}: {e}")

# Combine all merged subfolder DataFrames into a single DataFrame
if all_subfolder_data:
    consolidated_data = pd.concat(all_subfolder_data, ignore_index=True)
    consolidated_data.to_csv(output_file, index=False)
    print(f"Consolidation complete. File saved as {output_file}")
else:
    print("No data to process.")
    



In [3]:
# Import the pandas library
import pandas as pd

# Function to check and print the fields of a CSV file
def check_csv_fields(file_path):
    """
    Load a CSV file and print its fields (column names) along with metadata.

    Args:
        file_path (str): Path to the CSV file.
    """
    try:
        # Load CSV into a DataFrame without loading all data into memory
        df = pd.read_csv(file_path, nrows=0)

        # Print column names
        print("Field (Column) Names:")
        for column in df.columns:
            print(f"- {column}")

        # Print metadata
        print(f"\nNumber of fields: {len(df.columns)}")

    except Exception as e:
        print(f"Error loading CSV file: {e}")

# Example usage
# Replace 'your_file.csv' with the path to your CSV file
check_csv_fields('consolidated_data_2.csv')


Field (Column) Names:
- ACCESSION_NUMBER
- CUSIP
- VALUE
- SSHPRNAMT
- SSHPRNAMTTYPE
- PUTCALL
- FILINGMANAGER_NAME
- DATEREPORTED

Number of fields: 8


In [6]:
import pandas as pd

def delete_fields_from_csv(input_file, output_file, fields_to_delete):
    """
    Deletes specified fields (columns) from a CSV file and saves the updated data to a new file.

    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the updated CSV file.
        fields_to_delete (list): List of column names to delete from the CSV.

    Returns:
        None
    """
    try:
        # Load the CSV file
        df = pd.read_csv(input_file)

        # Check if the columns to delete exist in the DataFrame
        missing_fields = [field for field in fields_to_delete if field not in df.columns]
        if missing_fields:
            print(f"Warning: The following fields were not found in the CSV and will be ignored: {missing_fields}")

        # Drop the specified fields
        updated_df = df.drop(columns=fields_to_delete, errors='ignore')

        # Save the updated DataFrame to a new CSV file
        updated_df.to_csv(output_file, index=False)
        print(f"Updated CSV saved to '{output_file}'.")

    except Exception as e:
        print(f"Error processing the file: {e}")

# Example usage
input_file = 'consolidated_data_2.csv'  # Replace with your input file path
output_file = 'final_13f.csv'  # Replace with your desired output file path
fields_to_delete = ['ACCESSION_NUMBER']  # Replace with the fields/columns you want to delete

delete_fields_from_csv(input_file, output_file, fields_to_delete)

/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_1543/531398034.py:17: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Updated CSV saved to 'final_13f.csv'.


In [8]:
import pandas as pd

class YourClass:
    input_path = "final_13f.csv"

    def CUSIP_to_ticker(self, output_csv_path="institutional_data.csv"):
        """
        Reads institutional data, maps CUSIP codes to tickers using a reference CSV,
        removes invalid dates, and writes the resulting DataFrame to a CSV file.

        Args:
            output_csv_path (str): Path to save the resulting CSV file.

        Returns:
            str: Path to the generated CSV file.
        """
        # Reference CSV for CUSIP-to-Ticker mapping
        df_ref = pd.read_csv("cusip_tickers.csv", dtype=str)

        # Check if required columns exist
        if not {"CUSIP", "Ticker"}.issubset(df_ref.columns):
            raise ValueError("Reference CSV must have 'CUSIP' and 'Ticker' columns.")

        # Create a mapping dictionary
        cusip_to_ticker = df_ref.set_index("CUSIP")["Ticker"].to_dict()

        original_csv = self.input_path  # Path to the input file
        chunksize = 10**5  # Process data in chunks of 100,000 rows

        # Open the output file in write mode
        with pd.read_csv(original_csv, dtype=str, chunksize=chunksize) as reader, open(output_csv_path, "w") as output_file:
            for chunk_index, chunk in enumerate(reader):
                print(f"Processing chunk {chunk_index + 1}...")

                # Validate required columns
                if "CUSIP" not in chunk.columns or "DATEREPORTED" not in chunk.columns:
                    raise ValueError("Input CSV is missing 'CUSIP' or 'DATEREPORTED' columns.")

                # Map CUSIP to Ticker
                chunk["CUSIP"] = chunk["CUSIP"].map(cusip_to_ticker).fillna(chunk["CUSIP"])

                # Rename 'CUSIP' column to 'Ticker'
                chunk.rename(columns={"CUSIP": "Ticker", "DATEREPORTED": "date"}, inplace=True)

                # Convert 'date' column to datetime and drop invalid dates
                chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
                chunk.dropna(subset=["date"], inplace=True)  # Remove rows with invalid dates

                # Debugging: Print the first few rows of the current chunk
                print(chunk.head())

                # Write the processed chunk to the output CSV
                mode = "w" if chunk_index == 0 else "a"  # Write header only for the first chunk
                header = chunk_index == 0
                chunk.to_csv(output_file, index=False, mode=mode, header=header)

        print(f"Consolidated data has been saved to {output_csv_path}")
        return output_csv_path


# Create an instance of YourClass and call the method
analyzer = YourClass()
output_csv_path = analyzer.CUSIP_to_ticker()
print(f"Output CSV generated at: {output_csv_path}")


Processing chunk 1...
     Ticker VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
8364    AEF   237     26031            SH     NaN   
8365    ALB  4752     21703            SH     NaN   
8366   ABEV    14      5104            SH     NaN   
8367   AZUL  1330     65020            SH     NaN   
8368   BSBR   214     31916            SH     NaN   

                      FILINGMANAGER_NAME       date  
8364  Provida Pension Fund Administrator 2021-11-15  
8365  Provida Pension Fund Administrator 2021-11-15  
8366  Provida Pension Fund Administrator 2021-11-15  
8367  Provida Pension Fund Administrator 2021-11-15  
8368  Provida Pension Fund Administrator 2021-11-15  
Processing chunk 2...
           Ticker  VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
103387  485170302  77927    275000            SH     NaN   
103388  00434H108  17210    100000            SH     NaN   
103389  198287203   7967    418870            SH     NaN   
103390  485170302  27579    101904            SH     NaN   
103391  82028

/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")


Processing chunk 4...
Empty DataFrame
Columns: [Ticker, VALUE, SSHPRNAMT, SSHPRNAMTTYPE, PUTCALL, FILINGMANAGER_NAME, date]
Index: []
Processing chunk 5...
Empty DataFrame
Columns: [Ticker, VALUE, SSHPRNAMT, SSHPRNAMTTYPE, PUTCALL, FILINGMANAGER_NAME, date]
Index: []
Processing chunk 6...


/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")


           Ticker      VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
512376  12739A100   34368000   1565040            SH     NaN   
512377  67020Y100  815306000  14812964            SH     NaN   
519995  015351109   85873000    549621            SH     NaN   
519996  57772K101  188214000   2123121            SH     NaN   
519997  983919101   87151000    614741            SH     NaN   

       FILINGMANAGER_NAME       date  
512376    UBS OCONNOR LLC 2021-09-30  
512377    UBS OCONNOR LLC 2021-09-30  
519995    UBS OCONNOR LLC 2020-12-31  
519996    UBS OCONNOR LLC 2020-12-31  
519997    UBS OCONNOR LLC 2020-12-31  
Processing chunk 7...
           Ticker   VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
602287        VNO  360374   9364453            SH     NaN   
602288  485170302   10786     39855            SH     NaN   
602289  82028K200   20120    694645            SH     NaN   
602290  82028K200   40434   1392535            SH     NaN   
602291  82028K200    8062    277700            SH     

...

Processing chunk 500...
             Ticker  VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
49979303         MX   2266    108077            SH     NaN   
49979304         MX   1907     90923            SH     NaN   
49979305  G6610J209  22433    904212            SH     NaN   
49979306       SPWH  16101   1364485            SH     NaN   
49979307       SPWH  12958   1098117            SH     NaN   

                             FILINGMANAGER_NAME       date  
49979303  Kite Lake Capital Management (UK) LLP 2022-12-21  
49979304  Kite Lake Capital Management (UK) LLP 2022-12-21  
49979305  Kite Lake Capital Management (UK) LLP 2022-12-21  
49979306  Kite Lake Capital Management (UK) LLP 2022-12-21  
49979307  Kite Lake Capital Management (UK) LLP 2022-12-21  
Processing chunk 501...
         Ticker  VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
50003721    AEF    163     26031            SH     NaN   
50003722    ALB  41027    193626            SH     NaN   
50003723   ABEV     13      5104       

/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")


Processing chunk 509...
Empty DataFrame
Columns: [Ticker, VALUE, SSHPRNAMT, SSHPRNAMTTYPE, PUTCALL, FILINGMANAGER_NAME, date]
Index: []
Processing chunk 510...
             Ticker      VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
50961156  177376100  410865000   4228313            SH     NaN   
50961157  59001K100   67484000   1857505            SH     NaN   
50961158  727493108   42176000   1062886            SH     NaN   
50961159  78781P105  317289000   5062053            SH     NaN   
50961160  90041T108   58319000    775000            SH     NaN   

         FILINGMANAGER_NAME       date  
50961156    UBS OCONNOR LLC 2022-06-30  
50961157    UBS OCONNOR LLC 2022-06-30  
50961158    UBS OCONNOR LLC 2022-06-30  
50961159    UBS OCONNOR LLC 2022-06-30  
50961160    UBS OCONNOR LLC 2022-06-30  
Processing chunk 511...
             Ticker  VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
51060010  15912K100  42332   1980000            SH     NaN   
51060011  15912K100  44183   2110000            S

/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
/var/folders/3h/d02yvjrj1tn_s50y0pcr97240000gn/T/ipykernel_3407/1630401453.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")


Processing chunk 512...
         Ticker    VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL FILINGMANAGER_NAME  \
51183285   BKNG   947419    399103            SH     NaN        NORGES BANK   
51183286    BAH   116473   1467844            SH     NaN        NORGES BANK   
51183287    BWA    95771   2216402            SH     NaN        NORGES BANK   
51183288    SAM    34762     68194            SH     NaN        NORGES BANK   
51183289    BXP  1412619  13037554            SH     NaN        NORGES BANK   

               date  
51183285 2021-11-05  
51183286 2021-11-05  
51183287 2021-11-05  
51183288 2021-11-05  
51183289 2021-11-05  
Processing chunk 513...
             Ticker  VALUE SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
51234975  86738J106  12258   4155380            SH     NaN   

          FILINGMANAGER_NAME       date  
51234975  ECP ControlCo, LLC 2022-11-14  
Processing chunk 514...
Empty DataFrame
Columns: [Ticker, VALUE, SSHPRNAMT, SSHPRNAMTTYPE, PUTCALL, FILINGMANAGER_NAME, date]
Index: []


In [9]:
# Old code

"""
df_ref = pd.read_csv("cusip_tickers.csv", dtype=str)
cusip_to_ticker = df_ref.set_index("CUSIP")["Ticker"].to_dict()

# Check columns exist
if not {"CUSIP", "Ticker"}.issubset(df_ref.columns):
    raise ValueError("Reference CSV must have 'CUSIP' and 'Ticker' columns.")
    
# Convert to dict
   
def process_dataframe(df, cusip_to_ticker):
    """
    Maps CUSIP codes to tickers within a DataFrame and renames the column.

    Args:
        df (pd.DataFrame): The original DataFrame with a 'CUSIP' column.
        cusip_to_ticker (dict): Dictionary mapping CUSIP codes to tickers.

    Returns:
        pd.DataFrame: The updated DataFrame with 'CUSIP' replaced by 'Ticker'.
    """
    # Check if "CUSIP" exists in the DataFrame
    if "CUSIP" not in df.columns:
        raise ValueError("The DataFrame is missing a 'CUSIP' column.")

    # Map CUSIP to Ticker and handle missing mappings
    df["CUSIP"] = df["CUSIP"].map(cusip_to_ticker).fillna(df["CUSIP"])

    # Rename "CUSIP" column to "Ticker"
    df.rename(columns={"CUSIP": "Ticker"}, inplace=True)

    return df


# Process the DataFrame
processed_df = process_dataframe(df, cusip_to_ticker)

print(processed_df)
      
    """

         Ticker   VALUE  SSHPRNAMT SSHPRNAMTTYPE PUTCALL  \
0          AVNT  182885    3268729            SH     NaN   
1           EPD  811362   36947247            SH     NaN   
2           FUL  183213    2261888            SH     NaN   
3           TAC   27247    2450300            SH     NaN   
4          BABA    1330      11200            SH     NaN   
...         ...     ...        ...           ...     ...   
52597027    IWM     862       5088            SH     NaN   
52597028    IWR     578       8935            SH     NaN   
52597029    IVW    8733     144712            SH     NaN   
52597030    IVE    4500      32740            SH     NaN   
52597031    IJK    3538      55515            SH     NaN   

                FILINGMANAGER_NAME DATEREPORTED  
0              Marquard & Bahls AG          NaN  
1              Marquard & Bahls AG          NaN  
2              Marquard & Bahls AG          NaN  
3              Marquard & Bahls AG          NaN  
4                      Pariax